In [1]:
from __future__ import annotations
import polars as pl 
import inspect
import numpy as np

In [ ]:
import polars as pl
import random 

df = pl.scan_csv("../data/test.csv")
    
# Define polars custom functions to apply
def add_position_column(df:pl.LazyFrame):
    df = df.with_columns( 
        pl.when(pl.col('defensive_skill') > 50).then('CB')
        .when(pl.col('offensive_skill') > 50).then('FW')
        .otherwise('bench').alias("position")
    )
    return df

def add_squad_number_column(df:pl.LazyFrame):
    df = df.with_columns( 
        pl.when(pl.col('position') == 'CD').then(pl.lit(random.sample(range(2, 6), 1)[0], dtype=pl.Int8))
        .when(pl.col('position') == 'FW').then(pl.lit(random.sample(range(7, 19), 1)[0], dtype=pl.Int8))
        .otherwise('-').alias("squad_number")
    )
    return df

# Chain operations together using the pipe function


df.pipe(add_position_column)\
    .pipe(add_squad_number_column)\
    .write_json("pipe.json")


In [ ]:
f = open("pipe.json", "r")
json_str = f.read()
f.close()
df3 = pl.read_csv("../data/test.csv")
df2 = df3.lazy().from_json(json_str)
df2.collect()

In [ ]:
plan = pl.LazyFrame().from_json(json_str)
plan.write_json("pipe.json")

In [ ]:
df2.collect()

In [ ]:
test = ["a","b"]

"|".join(test)

In [ ]:
def test() -> list[str]:
    return ["a"]

test.__annotations__.get("return", "") == list[str]

In [ ]:
from typing import Callable, Concatenate

tt:Callable[[], list]
tt = test

In [ ]:
inspect.signature(tt).return_annotation == "list[str]"

In [ ]:
df = pl.DataFrame({
    "g": list(range(1000))
}).lazy()

# test = pl.Series("x",["a", "b"])


In [ ]:
join_df = pl.LazyFrame((list(range(1000)), list(i*2 for i in range(1000))), schema=["g", "g_mapped"])
join_df.collect()

In [ ]:
%%timeit 
df.with_columns(
    pl.col("g").map_dict({i:2*i for i in range(1000)})
).collect()

In [ ]:
%%timeit 
df.join(join_df, on = "g").with_columns(
    pl.col("g_mapped").alias("g")
).drop(columns=["g_mapped"]).collect()

In [ ]:
df.join(join_df, on = "g").with_columns(
    pl.col("g_mapped").alias("g")
).drop(columns=["g_mapped"]).write_json("test.json")

In [ ]:
df = pl.DataFrame(
    {
        "a": list(range(1000)),
    }
)

In [ ]:
from typing import Generator, Tuple, Any

mapping = {i:i for i in range(1000)}
test = mapping.copy()
cname = "a"
def create_map_expr(
        col_name:str
        , gen:Generator[Tuple[str, Any], None, None]
        , default:Any = None
) -> pl.Expr:

    '''
        Suppose you have a dictionary like d = {"a":1, "b":2}. Instead of doing pl.col("column").map_dict(d), you can 
        do pl.when(pl.col("column") == "a").then(1).otherwise(pl.when(pl.col("column") == "b").then(2).otherwise(default))
        instead. This function generators this expression for you from a generator that yields a key value pair.
    
    '''

    try:
        k, v = next(gen)
        return pl.when(pl.col(col_name) == k).then(v).otherwise(create_map_expr(col_name, gen))
    except:
        return pl.lit(default)

In [ ]:
%%timeit 
df.select(
    pl.col("a").map_dict(mapping)
)

In [ ]:
m = list(range(1000))
expr = create_map_expr(cname, zip(m, m))

In [ ]:
%%timeit 

df.select(
    expr
)

In [ ]:
df = pl.DataFrame({"a":[None, 1], "b":[1,2]})
df2 = pl.DataFrame({"a":[None, 1], "c":[3,4]})

df.join(df2, how="left", on = "a")

In [ ]:
# Compute error, because pl.col(c) returns columns of different len depending on unique values
df.select(
    pl.col(c).unique().sort() for c in df.columns
)

In [ ]:
cs = df.lazy().groupby(1).agg(
    pl.col(c).unique().sort() for c in df.columns
).select(
    pl.col(c) for c in df.columns
).collect().get_columns()

for c in cs:
    print(c[0])

In [ ]:
for f in df.partition_by("a"):
    print(f.shape)

In [ ]:
df.select(
    pl.col(c).n_unique() for c in df.columns
)

In [3]:
d = pl.DataFrame(
    {
        "id": ["a", "b", "c", "d"],
        'num': [1,2,3,4],
        "events": [["e1", "e2"], ["e1", "e2"], ["e1", "e3"], [None]],
        "bools":[True, True, False, True]
    }
)

In [5]:
d.select(pl.col("bools").sum() / 2)

bools
f64
1.5


In [ ]:
d.lazy().select(pl.col("num").mean()).collect().to_numpy()[0,0]

In [ ]:
d.select(
    pl.col("events").list.unique().list.lengths()
)

In [ ]:
from sklearn.datasets import make_classification

In [ ]:
orig_x, orig_y = make_classification(n_samples = 300_000, n_features = 50, n_informative = 25, n_redundant = 25)
df = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))

In [ ]:
features = df.columns
features.remove("target")

In [ ]:
x = df.select(features)
v = pl.Series([2]*len(features))

print(x.shape)
print(v.shape)

In [ ]:
%%timeit
x.select(
    pl.col(c) * v[i] for i, c in enumerate(x.columns)
).fold(lambda s1, s2: s1 + s2)

In [ ]:
%%timeit
x.select(
    sum(pl.col(c) * v[i] for i, c in enumerate(x.columns))
)

In [ ]:
w = pl.Series([2]*200)


In [ ]:
def gradient_descent(df:pl.DataFrame, features:list[str], target:str):
    
    x = df.select(features)
    y = df.select(target)

    weights = pl.Series([1]*len(features))
    const = 0.

    


In [ ]:
import polars as pl
from dsds.prescreen import describe_str
from dsds.transform import ScalingStrategy

In [ ]:
df = pl.read_csv("../data/advertising.csv")

In [ ]:
describe_str(df, words_to_count=["A"])